In [ ]:
import json
from dataclasses import dataclass, field, asdict

@dataclass
class ITLPerformance:
    e_id: int = -1
    name: str = ""
    scobility: int = 0
    comfort_zone: int = 0
    charts_played: int = 0

@dataclass
class ITLEntrant:
    itl2022: ITLPerformance = field(default_factory=ITLPerformance)
    itl2023: ITLPerformance = field(default_factory=ITLPerformance)

with open("scobility_itl2022.json", "r") as fp:
    itl2022_data = json.load(fp)
    
with open("scobility_itl2023_20230926.json", "r") as fp:
    itl2023_data = json.load(fp)

In [ ]:
itl2022_data['players']

In [ ]:
from collections import defaultdict

# GS id -> performances
entrants = {}

data = {
    "itl2022": itl2022_data,
    "itl2023": itl2023_data
}
for year in ["itl2022", "itl2023"]:
    entrant_song_counts = defaultdict(set)

    n_songs = len(data[year]['songs'])
    for s in data[year]['scores']:
        entrant_song_counts[s['e_id']].add(s['s_id'])

    for p in data[year]['players']:
        player_data = ITLEntrant()
        if p['g_id'] in entrants:
            player_data = entrants[p['g_id']]
        getattr(player_data, year).e_id = p['e_id']
        getattr(player_data, year).name = p['name']
        getattr(player_data, year).scobility = p['scobility']
        getattr(player_data, year).comfort_zone = p['comfort_zone']
        getattr(player_data, year).charts_played = len(entrant_song_counts[p['e_id']])/n_songs
        entrants[p['g_id']] = player_data

In [ ]:
entrants

In [ ]:
asdict(entrants[1])

In [ ]:
import csv

with open("scobility_combine.csv", 'w', newline='') as fp:
    fieldnames = ['groovestats_id']
    entrant_objects = {k: {'groovestats_id': k} for k in entrants.keys()}
    for year in ['itl2022', 'itl2023']:
        fieldnames += [year + '_' + fn for fn in ['name', 'scobility', 'comfort_zone', 'charts_played']]
        for k, v in entrants.items():
            d = asdict(v)
            for fn in ['name', 'scobility', 'comfort_zone', 'charts_played']:
                entrant_objects[k][year + '_' + fn] = d[year][fn]
    writer = csv.DictWriter(fp, fieldnames=fieldnames)
    writer.writeheader()
    for k, v in entrant_objects.items():
        writer.writerow(v)